In [2]:
import math
import numpy as np
import torch
from datasets import load_dataset
from transformers import RobertaForSequenceClassification, RobertaTokenizer, RobertaTokenizerFast, RobertaConfig, RobertaModelWithHeads
from transformers import Trainer, TrainingArguments, EvalPrediction
from transformers import DataCollatorForLanguageModeling
from transformers import RobertaTokenizer, RobertaForMaskedLM, RobertaModelWithHeads
from transformers import AdapterType
from sklearn.metrics import f1_score

In [3]:
def check_gpu():
    '''
    check gpu status
    '''
    try:
        print('GPU available:', torch.cuda.is_available())
        print(torch.cuda.device_count(), 'GPUs detected')
        print('Current GPU id:', torch.cuda.current_device())
        print('Current GPU Name:', torch.cuda.get_device_name(torch.cuda.current_device()))
    except:
        print('GPU not available')

In [4]:
def encode_batch(batch):
    '''
    Encodes a batch of input data using the model tokenizer
    '''
    return tokenizer(batch["text"], max_length=120, truncation=True, padding="max_length")
#     return tokenizer(batch["text"], padding=True, truncation=True)

In [5]:
check_gpu()

GPU available: True
1 GPUs detected
Current GPU id: 0
Current GPU Name: NVIDIA GeForce GTX 980 Ti


In [6]:
# dictionary for dataset, name: (classes, type of f1 score)
dataset_dict = {'chemprot': (13, 'micro'), 'rct': (5, 'micro'),
                'CI': (6, 'macro'), 'sciie': (7, 'm2cro'),
                'HN': (2, 'macro'), 'ag': (4, 'macro'),
                'amazon': (2, 'macro'), 'imdb': (2, 'macro')}

In [7]:
ds_name = 'amazon'
n_labels = dataset_dict[ds_name][0]
f1_type = dataset_dict[ds_name][1]

In [8]:
dataset = load_dataset(f'data_loaders/{ds_name}_data_loader.py')

Reusing dataset task_dataset (C:\Users\snow-\.cache\huggingface\datasets\task_dataset\task\1.0.0\d4dbb1ae1e5b21302597f18c62e58ab7f320999e2bdffea6d0514c3c329ad9ae)


In [9]:
tokenizer = RobertaTokenizerFast.from_pretrained('roberta-base')

In [18]:
model = RobertaModelWithHeads.from_pretrained('roberta-base', return_dict=True).to('cuda')
# load pretrained model
# model = RobertaForMaskedLM.from_pretrained('model/amazon_mlm.pt', return_dict=True).to('cuda')

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModelWithHeads: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaModelWithHeads from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModelWithHeads from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaModelWithHeads were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.embeddings.position_ids']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and infere

In [19]:
model.add_classification_head("classifier", num_labels=n_labels)

In [20]:
model.add_adapter("classifier", adapter_type=AdapterType.text_task, config="pfeiffer")

In [21]:
model.train_adapter(['classifier'])

In [22]:
model.set_active_adapters(["classifier"])

In [26]:
# test model output
model(**tokenizer("Hello, my dog is cute", return_tensors="pt").to('cuda'))

SequenceClassifierOutput(loss=None, logits=tensor([[0.0812, 0.0184]], device='cuda:0', grad_fn=<AddmmBackward>), hidden_states=None, attentions=None)

In [12]:
# model.add_adapter('language', AdapterType.text_lang)

In [13]:
# model.train_adapter('language')

In [27]:
dataset_encoded = dataset.map(encode_batch, batched=True, batch_size=512, remove_columns=["text"])
# tokenized_datasets = dataset.map(tokenizer, batched=True, num_proc=4, remove_columns=["text"])

Loading cached processed dataset at C:\Users\snow-\.cache\huggingface\datasets\task_dataset\task\1.0.0\d4dbb1ae1e5b21302597f18c62e58ab7f320999e2bdffea6d0514c3c329ad9ae\cache-337b3387bac896d2.arrow
Loading cached processed dataset at C:\Users\snow-\.cache\huggingface\datasets\task_dataset\task\1.0.0\d4dbb1ae1e5b21302597f18c62e58ab7f320999e2bdffea6d0514c3c329ad9ae\cache-55af9a60e66b6087.arrow
Loading cached processed dataset at C:\Users\snow-\.cache\huggingface\datasets\task_dataset\task\1.0.0\d4dbb1ae1e5b21302597f18c62e58ab7f320999e2bdffea6d0514c3c329ad9ae\cache-19abf50b9f1125cc.arrow


In [28]:
dataset_encoded

DatasetDict({
    train: Dataset({
        features: ['attention_mask', 'input_ids', 'labels'],
        num_rows: 115251
    })
    validation: Dataset({
        features: ['attention_mask', 'input_ids', 'labels'],
        num_rows: 5000
    })
    test: Dataset({
        features: ['attention_mask', 'input_ids', 'labels'],
        num_rows: 25000
    })
})

In [29]:
dataset_encoded.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])

In [15]:
# data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm_probability=0.15)

In [30]:
training_args = TrainingArguments(
    "test-classification",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    weight_decay=0.01,
)

In [31]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset_encoded["train"],
    eval_dataset=dataset_encoded["validation"]
)

In [32]:
trainer.train()

RuntimeError: grad can be implicitly created only for scalar outputs